# Week 5 Discussion: SVM: The Impact of Kernels

## Objectives

In this week's discussion, we will learn:
* Effect of different kernels on SVM performance
* Using embarrassingly parallel tools for grid search

## Sources

This discussion on SVMs is partly based on p. 79-90, 105-110 of "Foundations of Machine Learning" by Mehryar Mohri, Afshin Rostamizadeh, and Ameet Talwalkar. 

## Preparing the Data

Before importing the data, as usual, we import _pandas_ library to simplify data manipulation.

In [1]:
import pandas as pd

In this discussion, similar to previous discussions, we use _Hitters_ dataset which is the data collected from Major League Baseball (MLB) data from the 1986 and 1987 seasons. We want to predict the new league of each player at the end of the season based on their performance in the previous league. For more information on the dataset, please look it up!

In [2]:
#the data can be downloaded from "https://github.com/jcrouser/islr-python/blob/master/data/Hitters.csv"
df = pd.read_csv('Hitters.csv')

We now print out the dataset to gain more insight in it:

In [3]:
print (df.head(5))

              Player  AtBat  Hits  HmRun  Runs  RBI  Walks  Years  CAtBat  \
0     -Andy Allanson    293    66      1    30   29     14      1     293   
1        -Alan Ashby    315    81      7    24   38     39     14    3449   
2       -Alvin Davis    479   130     18    66   72     76      3    1624   
3      -Andre Dawson    496   141     20    65   78     37     11    5628   
4  -Andres Galarraga    321    87     10    39   42     30      2     396   

   CHits    ...      CRuns  CRBI  CWalks  League Division PutOuts  Assists  \
0     66    ...         30    29      14       A        E     446       33   
1    835    ...        321   414     375       N        W     632       43   
2    457    ...        224   266     263       A        W     880       82   
3   1575    ...        828   838     354       N        E     200       11   
4    101    ...         48    46      33       N        E     805       40   

   Errors  Salary  NewLeague  
0      20     NaN          A  
1     

For imputation, similar to the past week, we remove all the rows containing any missing values.

# Handling the Missing Data, Still the Easiest Version

This week, we will not remove the rows with missing values and we perform mean imputation. We perform that after train-test split.

In [4]:
df.dropna(inplace=True)

As discussed above, the _new leage_ column in the dataset will be our label and others will be our features. We select all the columns, except for _NewLeague_, as our features and select _NewLeague_ as our label.

In [5]:
X = df[['AtBat','Hits','HmRun','Runs','RBI','Walks','Years','CAtBat','CHits','CHmRun','CRuns','CRBI','CWalks','League','Division','PutOuts','Assists','Errors','Salary']]
y = df[['NewLeague']]

Then, we do one-hot encoding to get the final stage of features:

In [6]:
X_cat = X.select_dtypes(exclude=['int64', 'float64'])                                                                                                         
X_dog = X.select_dtypes(include=['int64', 'float64'])                                                                                                         
                                                                                                                                                              
X_cat = pd.get_dummies(X_cat)                                                                                                                                 
X = pd.concat([X_cat, X_dog], axis=1)   

As discussed last week, changing the name of the outputs to numbers from $0$ to $k-1$ with $k$ being the number of different classes we have:

In [7]:
NewLeague2number_dict = {
    'A':0,
    'N':1
}

y=y.replace({"NewLeague": NewLeague2number_dict})

## A Short Recap on SVM

In our linear classification problem, considering our input space a subset of $\mathbb{R}^N$ with $N\geq1$, our hypothesis set $H$ consists of hyperplanes which can be defined as:

$$
H = \{x\rightarrow sign(w.x+b): w \in \mathbb{R}^N,b \in \mathbb{R}\}
$$

A hypothesis of this form labels positively all the points lying on one side, while labeling the others negatively.

### General Non-separable Case

In the general case where the data points are non-separable, the constraints imposed on the model by each data point can be written as:
<br>
$$
\exists \xi_i \mid y_i(w.x_i+b)\geq 1-\xi_i
$$
Therefore, the optimization problem would be:
<br>
$$
\underset{w,b,\xi}{min} \;\; \frac{1}{2}{\|w\|}^2 + C\sum_{n=1}^{m}\xi_i \\
\text{subject to:} \;\; y_i(w.x_i+b)\geq 1-\xi_i \; \wedge  \; \xi_i \geq 0, \; i \in [m]
$$

After the solving this optimization problem, we will have the hypothesis solution $h$ as:
<br>
<br>
$$
\begin{cases}
  h(x)=sgn(\sum_{i=1}^{m}\alpha_iy_i(x_i.x)+b)\\    
  b = y_i - \sum_{j=1}^{m}\alpha_jy_j(x_j.x_i)   
\end{cases}
$$
Where for any support vector $x_i$, $w.x_i+b=y_i$.

Since PDS kernels implicitly define inner products, we can replace the innter products $x_j.x_i$ in formulas above with $K(x_j.x_i)$. Therefore, we will have:
<br>
<br>
$$
\begin{cases}
  h(x)=sgn(\sum_{i=1}^{m}\alpha_iy_iK(x_j.x_i)+b)\\    
  b = y_i - \sum_{j=1}^{m}\alpha_jy_jK(x_j.x_i)   
\end{cases}
$$

## Cross-validating Different Kernels with Different Arguments

In this week's discussion, we compare different kernels with different parameters. For each specific type of kernel, we determine the best value $C$ (in the equation above) in addition to the parameters specific to each kernel.

## Takes long? Make It <font color='red'>Parallel</font> !

Last time we saw a way of handling computations that take long: saving results on disk for future use. Now we see another way of reducing the computation load: parallelization. The computation about to be done here is cross-validating different values of parameters for different kernels. In our previous approach, there was a loop in which a model was trained and tested based on every value. Since different models are independant in the training/testing process, we intend to parallelize some of the computation. Parallelization can be done at different levels. Now we intend to do parallelization at cross-validation level, meaning that the K-fold cross validations for a kernell with all the different combinations parameters be performed at the same time. 
<br>
For implementing our parallelization, we define a function that performs the K-fold cross-validation on a kernel with specific values for parameters and outputs the average AUC. This is the function the execution of which is to be parallelized:

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Imputer


def cross_validate_single_kernel(X, y, kernel, n_folds, args):
    
    auc = 0
    skf=StratifiedKFold(n_splits=n_folds, shuffle=True)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index].values.ravel(), y.iloc[test_index].values.ravel()

        print ("  *")
        ##normalization
        X_train_min, X_train_max = X_train.min(), X_train.max()
        X_train = (X_train - X_train_min)/(X_train_max-X_train_min)
        X_test  = (X_test  - X_train_min)/(X_train_max-X_train_min)

        X_train = X_train[X_train.columns[~(X_train_min==X_train_max)]]
        X_test  = X_test [X_test.columns[~(X_train_min==X_train_max)]]

        ##train/test
        svm = SVC(C=args["c_val"], kernel=kernel, tol=1e-7, shrinking=False, gamma=args["gamma_val"], degree = args["d_val"])
        svm.fit(X_train, y_train)

        pred   =   svm.predict(X_test)
        auc   += roc_auc_score(y_test, pred)/n_folds
    
    return auc

The function below cross validates a specific kernel with a range of different values for the hyperparameters. Since it uses all the combinations of the values in the product, it first makes sure none of the vectors are empty because that would make the whole product empty. Then it uses functions _parallel_ and _delayed_ to create a number of parallel jobs. The output would be a vector of values returned by the parallel execution of the passed function with the same order of the argument vector. We then create another vector, containing the corresponding names. The output of the function would be a dictionary mapping the names to the corresponding results.

In [9]:
from joblib import Parallel, delayed
import multiprocessing 
from itertools import product

num_processes = 30

def cross_validate_kernels(X, y, kernel, n_folds, args):
    
    for key in args.keys():
        if len(args[key])==0:
            args[key]=[1]

    results = Parallel(n_jobs=num_processes)(delayed(cross_validate_single_kernel)\
              (X, y, kernel, n_folds,{"c_val":c_val, "gamma_val":gamma_val, "d_val":d_val})\
              for (c_val, gamma_val, d_val) in product(args["c_vals"], args["gamma_vals"], args["d_vals"]))
    
    names   = ["kernel:%s__log10_C:%s_log10_gamma:%s__degree:%s"%(kernel, str(np.log10(c_val)), str(np.log10(gamma_val)), str(d_val))\
              for (c_val, gamma_val, d_val) in product(args["c_vals"], args["gamma_vals"], args["d_vals"])]
    
    auc_dict = dict(zip(names, results))
        
    return auc_dict

## Saving Results

Similar to the previous week, we will save the models in the results folder. In the following piece of code, we first check if it exists, and if not, we will create it:

In [10]:
import os

rslt_addr = "./results/"
if not os.path.exists(rslt_addr):
    os.makedirs(rslt_addr)

In week 4, it was also discussed that the names of the files should be as descriptive as possible so they do not get mixed up. This function produces names for kernel-argument combinations using which the results are saved to files.

In [11]:
def name_creator(kernel, n_folds, args):

    name = "kernel%s__n_folds%d"%(kernel, n_folds)
    
    def describe_in_string(vec, is_degree=False):
        
        if len(vec)==0:
            return "NA"
        
        if not is_degree:
            vec = np.log10(vec)
            
        if len(vec)==1:
            return "%d"%vec[0]
    
        return "%d~%d"%(np.min(vec), np.max(vec))
    
    for arg in sorted(args.keys()):
        if arg!="d_vals":
            name += "__log10_%s%s"%(arg, describe_in_string(args[arg]))
        else:
            name += "__%s%s"%(arg, describe_in_string(args[arg], True))
    
    return name

We see an example here:

In [12]:
import numpy as np

kernel     = "poly"
n_folds    = 5
c_vals     = np.power(float(10), range(-10, 10 + 1))
gamma_vals = np.power(float(10), range(-10, 10 + 1))
d_vals     = range(1,5+1)

print ('\n'*1, name_creator(kernel, n_folds, {"d_vals":d_vals, "gamma_vals":gamma_vals, "c_vals":c_vals}), '\n'*2)


 kernelpoly__n_folds5__log10_c_vals-10~10__d_vals1~5__log10_gamma_vals-10~10 




In [14]:
import pickle
import os
import numpy as np

def load_it_or_compute_it (X, y, kernel, n_folds, args):
    
    rslt_dict_name = name_creator(kernel, n_folds, args)
    rslt_dict_addr = os.path.join(rslt_addr, rslt_dict_name)
    
    print (rslt_dict_name)
    
    if os.path.isfile(rslt_dict_addr):
        print ("FOUND!")
        with open(rslt_dict_addr,"rb") as rslt_dict_handle:
            rslt_dict = pickle.load(rslt_dict_handle)
    else:
        rslt_dict = cross_validate_kernels(X, y, kernel, n_folds, args)
        with open(rslt_dict_addr,"wb") as rslt_dict_handle:
            pickle.dump(rslt_dict, rslt_dict_handle)

    return rslt_dict

### RBF Kernel

The Radial Basis Function (RBF) can be written as:
<br>
$$
\LARGE
K(x, x') = e^{\gamma{\|x-x'\|}^2}
\normalsize
$$
<br>
So there will be two parameters to cross-validate: C, and $\gamma$.

In [15]:
c_vals     = np.power(float(10), range(-9, 9 + 1))
gamma_vals = np.power(float(10), range(-9, 9 + 1))
d_vals     = []


load_it_or_compute_it (X, y, "rbf", 5, {"c_vals":c_vals, "gamma_vals":gamma_vals, "d_vals":d_vals})

kernelrbf__n_folds5__log10_c_vals-9~9__d_valsNA__log10_gamma_vals-9~9
FOUND!


{'kernel:rbf__log10_C:-1.0_log10_gamma:-1.0__degree:1': 0.9315755336617406,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-2.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-3.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-4.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-5.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-6.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-7.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-8.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:-9.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:0.0__degree:1': 0.9281986863711,
 'kernel:rbf__log10_C:-1.0_log10_gamma:1.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:2.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:3.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:4.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:5.0__degree:1': 0.5,
 'kernel:rbf__log10_C:-1.0_log10_gamma:6.0__degree:1': 0.5,
 'ke

### Linear Kernel

The linear kernel can be written as:
<br>
$$
\LARGE
K(x, x') = <x.x'>
\normalsize
$$
<br>
So there will be only one parameter to cross-validate: C.

In [16]:
c_vals     = np.power(float(10), range(-7, 7 + 1))
gamma_vals = []
d_vals     = []


load_it_or_compute_it (X, y, "linear", 5, {"c_vals":c_vals, "gamma_vals":gamma_vals, "d_vals":d_vals})

kernellinear__n_folds5__log10_c_vals-7~7__d_valsNA__log10_gamma_valsNA
FOUND!


{'kernel:linear__log10_C:-1.0_log10_gamma:0.0__degree:1': 0.9409770114942528,
 'kernel:linear__log10_C:-2.0_log10_gamma:0.0__degree:1': 0.9409414340448822,
 'kernel:linear__log10_C:-3.0_log10_gamma:0.0__degree:1': 0.5,
 'kernel:linear__log10_C:-4.0_log10_gamma:0.0__degree:1': 0.5,
 'kernel:linear__log10_C:-5.0_log10_gamma:0.0__degree:1': 0.5,
 'kernel:linear__log10_C:-6.0_log10_gamma:0.0__degree:1': 0.5,
 'kernel:linear__log10_C:-7.0_log10_gamma:0.0__degree:1': 0.5,
 'kernel:linear__log10_C:0.0_log10_gamma:0.0__degree:1': 0.9407827038861521,
 'kernel:linear__log10_C:1.0_log10_gamma:0.0__degree:1': 0.9406321839080459,
 'kernel:linear__log10_C:2.0_log10_gamma:0.0__degree:1': 0.9410919540229885,
 'kernel:linear__log10_C:3.0_log10_gamma:0.0__degree:1': 0.9409770114942528,
 'kernel:linear__log10_C:4.0_log10_gamma:0.0__degree:1': 0.9407115489874109,
 'kernel:linear__log10_C:5.0_log10_gamma:0.0__degree:1': 0.9407827038861521,
 'kernel:linear__log10_C:6.0_log10_gamma:0.0__degree:1': 0.94074712

### Polynomial Kernel

The polynomial kernel can be written as:
<br>
$$
\LARGE
K(x, x') = (\gamma<x.x'>)^d
\normalsize
$$
<br>
So there will be three parameters to cross-validate: C, $\gamma$, and $d$.

In [ ]:
c_vals     = np.power(float(10), range(-5, 5 + 1))
gamma_vals = np.power(float(10), range(-5, 5 + 1))
d_vals     = range( 1, 10 + 1)


load_it_or_compute_it (X, y, "poly", 5, {"c_vals":c_vals, "gamma_vals":gamma_vals, "d_vals":d_vals})

kernelpoly__n_folds5__log10_c_vals-5~5__d_vals1~10__log10_gamma_vals-5~5


### Sigmoid Kernel

The polynomial kernel can be written as:
<br>
$$
\LARGE
K(x, x') = tgh(\gamma<x.x'>)
\normalsize
$$
<br>
So there will be only one parameter to cross-validate: C, $\gamma$, and $d$.

In [17]:
c_vals     = np.power(float(10), range(-5, 5 + 1))
gamma_vals = np.power(float(10), range(-5, 5 + 1))
d_vals     = []


load_it_or_compute_it (X, y, "sigmoid", 5, {"c_vals":c_vals, "gamma_vals":gamma_vals, "d_vals":d_vals})

kernelsigmoid__n_folds5__log10_c_vals-5~5__d_valsNA__log10_gamma_vals-5~5
FOUND!


{'kernel:sigmoid__log10_C:-1.0_log10_gamma:-1.0__degree:1': 0.9319885057471264,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:-2.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:-3.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:-4.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:-5.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:0.0__degree:1': 0.7809802955665024,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:1.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:2.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:3.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:4.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-1.0_log10_gamma:5.0__degree:1': 0.5005952380952382,
 'kernel:sigmoid__log10_C:-2.0_log10_gamma:-1.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-2.0_log10_gamma:-2.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-2.0_log10_gamma:-3.0__degree:1': 0.5,
 'kernel:sigmoid__log10_C:-2.0_log10_gamma:-4.0__de